In [3]:
import pandas as pd
import requests
import numpy as np
from operator import delitem
import csv
from apscheduler.schedulers.background import BackgroundScheduler

def update_game_results():
    url = "https://v1.baseball.api-sports.io/games?league=1&season=2023"

    payload={}
    headers = {
      'x-rapidapi-key': '1ae57e7a748cd2bb05d6fd7321c4747a',
      'x-rapidapi-host': 'v1.baseball.api-sports.io'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    data = response.json()

    y = data.get('response')

    def game_results(num):

        y = data.get('response')
        x = (y[num]).get('teams')

        a = x.get('away')
        b = x.get('home')

        away_team = a.get('name')
        home_team = b.get('name')


        z = (y[num]).get('scores')

        c = z.get("home")
        d = z.get('away')

        home_score = c.get('total')
        away_score = d.get('total')

        date = (y[num]).get('date')



        return(date, home_team,home_score, away_team, away_score)

    blank = pd.DataFrame(list())
    blank.to_csv('mlb_game_data_2023.csv')

    csv_file = open("mlb_game_data_2023.csv", 'w')

    csv_writer = csv.writer(csv_file, delimiter=",")

    for i in range(len(y)):
        csv_writer.writerow(game_results(i))

    csv_file.close()

    df = pd.read_csv("mlb_game_data_2023.csv")

    df.columns = ['date','home_team','home_runs','away_team','away_runs']
    df = df[468:].reset_index(drop=True).dropna(axis=0) # drop spring training and unplayed games

    def format_date(string):
        string = string[0:10]
        return string

    df['date'] = df['date'].apply(format_date)

    df.to_csv("/home/luke/projects/blog/mlb_game_data_2023.csv",sep=',')

# Create a background scheduler
scheduler = BackgroundScheduler()

# Schedule the update_game_results function to run every 24 hours
scheduler.add_job(update_game_results, 'interval', hours=24)

# Start the scheduler
scheduler.start()

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
